# 스포티지 다중 내부 크롤링

In [140]:
from selenium import webdriver
from selenium.webdriver.common.by import By
import time
from bs4 import BeautifulSoup
import pandas as pd
import chromedriver_autoinstaller
import re

# 크롬 드라이버 자동 설치 및 설정
chromedriver_autoinstaller.install()
options = webdriver.ChromeOptions()
options.add_argument('--disable-extensions')
options.add_argument('--disable-gpu')
options.add_argument('--disable-software-rasterizer')
options.add_argument('--no-sandbox')
options.add_argument('--disable-dev-shm-usage')
driver = webdriver.Chrome(options=options)

# 데이터베이스 불러오기
data = {
    "Model": ["캐스퍼 결함"]
}

df = pd.DataFrame(data)

# 다양한 형식의 날짜와 시간을 추출하는 함수
def extract_date_time(soup):
    patterns = [
        r"입력\s?:\s?(\d{4}/\d{2}/\d{2}\s\d{2}:\d{2})",
        r"수정:\s?(\d{4}/\d{2}/\d{2}\s\d{2}:\d{2})",
        r"(\d{4}\.\d{2}\.\d{2})\s?\|\s?(\d{2}:\d{2}:\d{2})",
        r"입력\s(\d{4}\.\d{2}\.\d{2}\s\d{2}:\d{2})",
        r"승인\s(\d{4}\.\d{2}\.\d{2}\s\d{2}:\d{2})",
        r"(\d{4}\.\d{2}\.\d{2}\s\d{2}:\d{2})",
        r"(\d{4}-\d{2}-\d{2}\s\d{2}:\d{2})",
        r"입력\s(\d{4}-\d{2}-\d{2}\s\d{2}:\d{2})",
        r"승인\s(\d{4}-\d{2}-\d{2}\s\d{2}:\d{2})",
        r"기사승인\s(\d{4}\.\s?\d{2}\.\s?\d{2}\.\s?\d{2}:\d{2})",
        r"수정\s(\d{4}\.\d{2}\.\d{2}\s\d{2}:\d{2})",
        r"(\d{4}\.\d{2}\.\d{2}\s오전\s\d{2}:\d{2})",
        r"(\d{4}\.\d{2}\.\d{2}\s오후\s\d{2}:\d{2})",
        r"(\d{4}\.\d{2}\.\d{2}\s\d{2}:\d{2}:\d{2})",
        r"datetime=\"(\d{4}-\d{2}-\d{2}T\d{2}:\d{2}:\d{2})\"",
        r"기사입력\s+(\d{4}-\d{2}-\d{2}\s\d{2}:\d{2})",
        r"<time\s[^>]*datetime=\"(\d{4}-\d{2}-\d{2}T\d{2}:\d{2})\"",
        r"<div\sclass=\"date\">\s?(\d{4}\.\d{2}\.\d{2})\s?(오전|오후)?\s?(\d{2}:\d{2})"
    ]
    for pattern in patterns:
        match = re.search(pattern, soup.get_text())
        if match:
            date_time_str = match.group(1)
            # 오전/오후 처리를 위해 추가된 코드
            if "오전" in date_time_str:
                date_time_str = date_time_str.replace("오전", "").strip()
            elif "오후" in date_time_str:
                date_time_str = date_time_str.replace("오후", "").strip()
                dt = datetime.strptime(date_time_str, "%Y.%m.%d %H:%M")
                dt += timedelta(hours=12)
                return dt.strftime("%Y-%m-%d %H:%M")
            try:
                dt = datetime.strptime(date_time_str, "%Y.%m.%d %H:%M")
            except ValueError:
                try:
                    dt = datetime.strptime(date_time_str, "%Y-%m-%dT%H:%M:%S")
                except ValueError:
                    dt = datetime.strptime(date_time_str, "%Y-%m-%d %H:%M")
            return dt.strftime("%Y-%m-%d %H:%M")
    return ""

# 다양한 형식의 기사 본문을 추출하는 함수 (전체 페이지 소스를 반환)
def extract_article_content(soup):
    return soup.get_text(strip=True)

# 기사 페이지에서 데이터 추출 함수
def get_article_data(driver, url):
    try:
        driver.set_page_load_timeout(20)  # 페이지 로드 타임아웃을 20초로 설정
        driver.get(url)
        time.sleep(1)
        soup = BeautifulSoup(driver.page_source, 'html.parser')
        
        # 날짜 및 시간 추출
        date_time = extract_date_time(soup)
        
        # 기사 내용 추출
        article_content = extract_article_content(soup)
        
        return date_time, article_content
    except Exception as e:
        print(f"페이지 접근 실패: {url}, 에러: {e}")
        return "", ""

# 검색 페이지에서 데이터 추출 함수
def get_page_data(driver):
    soup = BeautifulSoup(driver.page_source, 'html.parser')
    articles = soup.find_all('li', class_='bx')
    data = []

    for article in articles:
        title_tag = article.find('a', class_='news_tit')
        date_tag = article.find('span', class_='info')
        link_tag = title_tag.get('href') if title_tag else None
        
        if title_tag and date_tag:
            title = title_tag.get('title')
            date = date_tag.text.strip()
            link = link_tag.strip()
            date_time, article_content = get_article_data(driver, link)
            data.append({'Title': title, 'Date': date, 'Link': link, 'DateTime': date_time, 'Content': article_content})
    
    return data

# 끝까지 스크롤하고 마지막 게시물을 찾는 함수
def scroll_to_bottom(driver):
    last_height = driver.execute_script("return document.body.scrollHeight")
    while True:
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        time.sleep(0.1)  # 페이지 로딩을 기다림
        new_height = driver.execute_script("return document.body.scrollHeight")
        if new_height == last_height:
            break
        last_height = new_height

# 각 항목에 대해 크롤링 수행
for index, row in df.iterrows():
    model = row['Model']
    start_date = '2024.01.01'
    end_date = '2024.12.31'   
    query = f"{model}"
    
    # URL 생성
    base_url = "https://search.naver.com/search.naver"
    params = f"?where=news&query={query}&sm=tab_opt&sort=1&photo=0&field=0&pd=3&ds={start_date}&de={end_date}&docid=&related=0&mynews=0&office_type=0&office_section_code=0&news_office_checked=&nso=so%3Add%2Cp%3Afrom{start_date.replace('.','')}to{end_date.replace('.','')}&is_sug_officeid=0&office_category=0&service_area=0"
    url = base_url + params

    # 첫 페이지 로드
    driver.get(url)
    time.sleep(1)  # 페이지 로딩을 기다림

    # 스크롤하여 끝 게시물 찾기
    scroll_to_bottom(driver)

    # 마지막 게시물 정보 출력
    soup = BeautifulSoup(driver.page_source, 'html.parser')
    articles = soup.find_all('li', class_='bx')
    if articles:
        last_article = articles[-1]
        last_title = last_article.find('a', class_='news_tit').get('title') if last_article.find('a', 'news_tit') else "제목 없음"
        last_date = last_article.find('span', 'info').text.strip() if last_article.find('span', 'info') else "날짜 없음"
        print(f"마지막 게시물 제목: {last_title}")
        print(f"마지막 게시물 날짜: {last_date}")
    
    # 데이터 개수 출력
    total_articles = len(articles)
    print(f"총 게시물 개수: {total_articles}")

    # 데이터를 저장할 리스트
    all_data = []

    # 데이터 추출
    page_data = get_page_data(driver)
    all_data.extend(page_data)
    
    # 데이터를 DataFrame으로 변환
    result_df = pd.DataFrame(all_data).drop_duplicates().reset_index(drop=True)

    # 파일 이름 생성
    model_cleaned = model.replace("/", "").replace(" ", "_").replace("(", "").replace(")", "")
    file_name = f"/Users/admin/softeer/project/{model_cleaned}.xlsx"

    # DataFrame을 엑셀 파일로 저장
    result_df.to_excel(file_name, index=False)
    print(f"데이터가 '{file_name}' 파일에 저장되었습니다.")

# 크롤링 완료 후 브라우저 종료
driver.quit()


마지막 게시물 제목: [월간車車車] '청룡의 해' 날개 펴는 기아의 꿈은?
마지막 게시물 날짜: 2024.01.09.
총 게시물 개수: 68
페이지 접근 실패: https://zdnet.co.kr/view/?no=20240710140814, 에러: time data '2024/07/10 16:08' does not match format '%Y-%m-%d %H:%M'
페이지 접근 실패: https://www.moneys.co.kr/article/2024050811032279985, 에러: Message: timeout: Timed out receiving message from renderer: 19.787
  (Session info: chrome=127.0.6533.89)
Stacktrace:
0   chromedriver                        0x0000000104fe1088 cxxbridge1$str$ptr + 1887276
1   chromedriver                        0x0000000104fd9764 cxxbridge1$str$ptr + 1856264
2   chromedriver                        0x0000000104be882c cxxbridge1$string$len + 88524
3   chromedriver                        0x0000000104bd3df4 cxxbridge1$string$len + 3988
4   chromedriver                        0x0000000104bd3b7c cxxbridge1$string$len + 3356
5   chromedriver                        0x0000000104bd1954 core::str::slice_error_fail::he7b2aa4898bc357e + 59976
6   chromedriver                        0x00000001